In [ ]:
import os
from glob import glob
import numpy as np
from read_dataset import build_df
from utils import CFG
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
from tqdm import tqdm
from torch.utils.data import DataLoader
from datapreprocess import preprocess_image

import seaborn as sns
from sklearn.metrics import *
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torchvision
from utils import CFG2


# Parameters
params = {"batch_size": 8, "shuffle": True, "num_workers": 4}
NUM_CLASSES = 11


images_folder  = CFG2.imgs_path
labels_folder = CFG2.labels_path

In [1]:
from models import CNN, tinyCNN,NeuralNet, get_pretrained_model
from helper_tools import reader, evalution_metrics, validate_model, get_default_device, trainCustomModel, plot_model_history
from helper_tools import CustomDatasetV2 as CustomDataset

In [ ]:
train_data, validation_data, test_data = reader(images_folder, labels_folder)

In [ ]:
train_data[0][:5], train_data[1][:5]

In [ ]:
transform  = preprocess_image
train_dataset  = CustomDataset(train_data, transform_fn=transform)
validation_dataset = CustomDataset(validation_data, transform_fn=transform)

test_dataset = CustomDataset(test_data, transform_fn=transform)
np.max(np.array(train_dataset[9][0]))

In [ ]:
np.min(np.array(train_dataset[9][0]))

In [ ]:
train_dataset[0][0].shape

In [ ]:
DEVICE = get_default_device()

loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
EPOCHS = 2

# Parameters
params = {"batch_size": 8, "shuffle": True, "num_workers": 4}
max_epochs = 100
NUM_CLASSES = 11
RESNET_OUT_FEATURES = 1000

In [ ]:

# Dataloaders
train_dataloader = DataLoader(
    train_dataset, params["batch_size"], num_workers=params["num_workers"], shuffle=params["shuffle"],
)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, params['batch_size'],num_workers=params['num_workers'])
test_dataloader = DataLoader(
    test_dataset, params["batch_size"], num_workers=params["num_workers"]
)

In [ ]:
resnetModel  = get_pretrained_model().to(DEVICE)

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[1][0].shape

In [ ]:
# plt.imshow(train_dataset[0][0].permute(1,2,0))

In [ ]:

def train_tiny_cnn(model, train_dataloader,loss_fn, epochs=30, learning_rate=0.001, device='cpu'):
    """Accepts feature from resnet and yolo object detection cropped iamge(s) 
    as features to train an accurate cnn classifier.
    """
    # custom_cnn_classifier = tinyCNN(3, 32, 11).to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

    training_losses = []
    training_accs = []

    for epoch in range(1, epochs+1):
        number_of_batches = 0
        epoch_loss_values = 0.0
        epoch_accs = 0.0
        for index, (X, y) in enumerate(tqdm(train_dataloader)):
            
            X  = Variable(X, requires_grad=True).to(device)
            
            y  = y.to(device)

            # predict using resnet
            # resnet_X = resnet_model(X)
            preds = model(X)
            # print("preds",preds)
        
            loss = loss_fn(preds, y).to(device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            probs = torch.log_softmax(preds, dim=1)
            predicted_labels = torch.argmax(probs, dim=1)
            # print("predicted_labels",predicted_labels)

            # acc
            epoch_accs += accuracy_score(y.detach().cpu(),predicted_labels.detach().cpu())
            epoch_loss_values += loss.item()

            number_of_batches += 1

            # acc, accuracy
        batch_acc, batch_loss = epoch_accs / \
            number_of_batches, epoch_loss_values / number_of_batches
        training_losses.append(batch_loss)
        training_accs.append(batch_acc)

        print("Epoch:{}/{}, acc={:.3f}%, loss={:.3f}".format(epoch, epochs, batch_acc*100, batch_loss))

    print("Learning Finished!")

    return model, training_accs, training_losses

In [ ]:
filters  = 32
in_channels  = 3

resnetModel = get_pretrained_model().to(DEVICE)
yolov7Model  = None #get_yolov7_model().to(DEVICE)
custom_cnn_model  = CNN(in_channels, filters, NUM_CLASSES).to(DEVICE)

learning_rate = 0.001
optimizer = torch.optim.Adam(params=custom_cnn_model.parameters(), lr=learning_rate)


In [ ]:
custom_model, training_accs, training_losses = trainCustomModel(resnetModel, yolov7Model, custom_cnn_model, train_dataloader, optimizer,loss_fn, epochs=2, learning_rate=0.001, device=DEVICE)

In [ ]:
# train_tiny_cnn_model = tinyCNN(in_channels, filters, NUM_CLASSES).to(DEVICE)

In [ ]:

resnet_152  = get_pretrained_model().to(DEVICE)
neural_net_model = NeuralNet().to(DEVICE)
finetuned_resnet152_model  = nn.Sequential(resnet_152, neural_net_model).to(DEVICE)

In [ ]:
finetuned_resnet152_model, training_accs, training_losses = train_tiny_cnn(finetuned_resnet152_model, train_dataloader,loss_fn, epochs = 50, device=DEVICE)

In [ ]:
# training_accs, training_losses = train_tiny_cnn(train_dataloader,loss_fn, epochs=50, learning_rate=0.001, device=DEVICE)

In [ ]:
plot_model_history(training_accs, training_losses)

In [ ]:
test_labels, test_predictions  = validate_model(finetuned_resnet152_model, test_dataloader, device=DEVICE)

In [ ]:
evalution_metrics(test_labels, test_predictions)

In [ ]:
path = os.path.join(os.getcwd(),'models_weights.pth')

In [ ]:
torch.save(finetuned_resnet152_model.state_dict(), path)

In [ ]:
# customCNNModel.state_dict()

In [ ]:
# loaded_model.state_dict()

# model = CNN(3, 32, NUM_CLASSES).to(DEVICE)
# model.load_state_dict(torch.load(path))
# model.eval()

In [ ]:
# model.state_dict()


In [ ]:

validate_model(custom_cnn_classifier, test_dataloader)
